# Setup

In [15]:
%cd  /home/tlm/Work/FYP-22-23/weee-preprocess

/home/tlm/Work/FYP-22-23/weee-preprocess


In [16]:
# %pip install neurokit2 --quiet

In [17]:
import os
import datetime
import numpy as np
import pandas as pd
import neurokit2 as nk

In [18]:
DATASET_DIR = "./data/v1"
SAMPLING_RATE = 4 # Hz

# Prepare data

In [37]:
from utils.dataset import StudyInfoEncoder, DatasetVersion1
from utils.naming import standardize_column_names
from utils.resample import upsample

In [38]:
DatasetVersion1(DATASET_DIR).get(1, 'MUSE', 'EEG')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/tlm/.local/share/virtualenvs/weee-preprocess-I3n8R8lm/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_10277/539745384.py", line 1, in <module>
    DatasetVersion1(DATASET_DIR).get(1, 'MUSE', 'EEG')
  File "/home/tlm/Work/FYP-22-23/weee-preprocess/utils/dataset.py", line 86, in get
TypeError: DatasetVersion1.__muse_eeg() takes 3 positional arguments but 4 were given

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tlm/.local/share/virtualenvs/weee-preprocess-I3n8R8lm/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/tlm/.local/share/virtualenvs/weee-preprocess-I3n8R8lm/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1118, in structured_tra

# Process

In [34]:
df.head()

,timestamp,alpha,beta,delta,gamma,theta
0,2021-12-03 16:54:57.575885056+00:00,171.648346,802.637390,784.505493,241.355316,684.175842
1,2021-12-03 16:54:57.576206080+00:00,1550.073242,825.604370,788.131897,1573.443237,835.677673
2,2021-12-03 16:54:57.576272128+00:00,0.000000,787.325989,784.505493,0.000000,776.446899
3,2021-12-03 16:54:57.576153088+00:00,959.377319,798.205139,785.714294,942.454224,763.553101
4,2021-12-03 16:54:57.575933952+00:00,89.047623,811.904785,788.131897,72.930405,783.699646


# Windowing